In [1]:
import glob
import os, re
import tensorflow as tf
from sklearn.model_selection import train_test_split
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [2]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])                                   ", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence


# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue

    if len(sentence) > 17 : continue    
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> it goes like this <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> she tied you <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> in every word <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> and even though <end>']

In [3]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다

def tokenize(corpus):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)
    
#     tensor=[]
#     for i in range(tensor2):
#         print(len(tensor2[0]))
# #         if len([i]) < 15:
# #             tensor.append(i)
# #             print(tensor)
  
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    #print(tensor,tokenizer)

    
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)


src_input = tensor[:,:-1]
trg_input = tensor[:,1:]

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                         trg_input,
                                                         test_size=0.2,
                                                         random_state=1)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)


BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE


VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, trg_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

Source Train: (15303, 10)
Target Train: (15303, 10)


<BatchDataset shapes: ((256, 10), (256, 10)), types: (tf.int32, tf.int32)>

In [4]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)


# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)



optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)



Epoch 1/30
74/74 [==============================] - 11s 116ms/step - loss: 3.0470
Epoch 2/30
74/74 [==============================] - 9s 120ms/step - loss: 1.9560
Epoch 3/30
74/74 [==============================] - 9s 121ms/step - loss: 1.8571
Epoch 4/30
74/74 [==============================] - 9s 123ms/step - loss: 1.8077
Epoch 5/30
74/74 [==============================] - 9s 124ms/step - loss: 1.7640
Epoch 6/30
74/74 [==============================] - 9s 122ms/step - loss: 1.7181
Epoch 7/30
74/74 [==============================] - 9s 122ms/step - loss: 1.6734
Epoch 8/30
74/74 [==============================] - 9s 119ms/step - loss: 1.6297
Epoch 9/30
74/74 [==============================] - 9s 119ms/step - loss: 1.5888
Epoch 10/30
74/74 [==============================] - 9s 120ms/step - loss: 1.5467
Epoch 11/30
74/74 [==============================] - 9s 121ms/step - loss: 1.5075
Epoch 12/30
74/74 [==============================] - 9s 122ms/step - loss: 1.4710
Epoch 13/30
74/74 [=====

'<start> you re the one <end> '

In [13]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

'<start> i love you <end> '